In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt


# Тут идет калибровка на шахматную доску

chessboard_size = (10, 7) 

objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)


objpoints = []  
imgpoints = [] 

image_dir = '/Users/aleksandrbogdanov/Desktop/2М 1 семестр/Comp vision Shokurov/HW4 - Map calibrate/ChessImages'
images = [img for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]

for image_name in images:
    img = cv2.imread(os.path.join(image_dir, image_name))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)


        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [2]:
# С помощью findContours отыщем точки пересечения - corners 

image_path = '/Users/aleksandrbogdanov/Desktop/2М 1 семестр/Comp vision Shokurov/HW4 - Map calibrate/Map1_agles.jpg'
img = cv2.imread(image_path)

hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Определяем диапазон зелёного цвета в HSV
lower_green = np.array([35, 50, 50])  # Нижняя граница 
upper_green = np.array([85, 255, 255])  # Верхняя граница

# Создаем маску для зелёного цвета
mask = cv2.inRange(hsv_image, lower_green, upper_green)

contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

circles_coords = []


for contour in contours:
    # Находим минимальную окружность для каждого контура
    (x, y), radius = cv2.minEnclosingCircle(contour)
    
    if 5 <= radius <= 40:
        center = (int(x), int(y))
        radius = int(radius)
        
        circles_coords.append([int(x), int(y)]) 
        

        cv2.circle(img, center, radius, (0, 0, 255), 2) 
        
circles_coords = np.array(circles_coords)
# Отбирается один нежелательный круг - его уберем 
circle_coords = circles_coords[:-1]

# Отобразим центры окр-ей
for center in circle_coords:
    print(f"Circle center: {center[0], center[1]}")

cv2.imshow('Image with Green Circles', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Circle center: (449, 782)
Circle center: (519, 775)
Circle center: (599, 765)
Circle center: (449, 693)
Circle center: (514, 678)
Circle center: (594, 663)
Circle center: (449, 607)
Circle center: (514, 589)
Circle center: (590, 566)


In [5]:
# Нарисуем оси - перпендикулярные друг другу

def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255, 0, 0), 5)  # Ось X (синий)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0, 255, 0), 5)  # Ось Y (зеленый)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0, 0, 255), 5)  # Ось Z (красный)
    return img



objp = np.zeros((3*3,3), np.float32)
objp[:,:2] = np.mgrid[0:3,0:3].T.reshape(-1,2)
axis = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, 3]]).reshape(-1, 3) 

# Параметры камеры 
mtx = camera_matrix
dist = dist_coeffs

image_path = '/Users/aleksandrbogdanov/Desktop/2М 1 семестр/Comp vision Shokurov/HW4 - Map calibrate/Map1_agles.jpg'
img = cv2.imread(image_path)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


# Несмотря на то, что corners выше - одномерный массив, запишем его так 

circle_coords = np.array(([[449, 782],
       [519, 775],
       [599, 765],
       [449, 693],
       [514, 678],
       [594, 663],
       [449, 607],
       [514, 589],
       [590, 566]]), dtype=np.float32)



criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# corners2 = cv2.cornerSubPix(gray, circle_coords, (11, 11), (-1, -1), criteria)


retval, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, circle_coords, mtx, dist)


        # Печать для отладки
print("rvecs:", rvecs.ravel())
print("tvecs:", tvecs.ravel())
    

imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

img = draw(img, circle_coords, imgpts)
    
    # Отображение результата
cv2.imshow('3D Axes on Chessboard', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


rvecs: [ 2.6984636  -0.15687313 -1.04202281]
tvecs: [-0.36850534  1.64669871 10.7141859 ]


In [7]:
#Рисуем пирамиды

def draw_pyramid(img, base_center, base_size, height, rvecs, tvecs, camera_matrix, dist_coeffs):

    # Координаты вершин основания (квадрат)
    half_size = base_size / 2
    base_points = np.array([
        [base_center[0] - half_size, base_center[1] - half_size, base_center[2]],
        [base_center[0] + half_size, base_center[1] - half_size, base_center[2]],
        [base_center[0] + half_size, base_center[1] + half_size, base_center[2]],
        [base_center[0] - half_size, base_center[1] + half_size, base_center[2]],
    ])

    # Вершина пирамиды
    apex = np.array([[base_center[0], base_center[1], base_center[2] + height]])
    pyramid_points = np.vstack([base_points, apex])

    # Проекция точек пирамиды
    imgpts, jac = cv2.projectPoints(pyramid_points, rvecs, tvecs, camera_matrix, dist_coeffs)
    imgpts = imgpts.reshape(-1, 2).astype(int)

    # основание (квадрат)
    for i in range(4):
        cv2.line(img, tuple(imgpts[i]), tuple(imgpts[(i + 1) % 4]), (0, 0, 0), 2)

    # ребра от основания к вершине
    for i in range(4):
        cv2.line(img, tuple(imgpts[i]), tuple(imgpts[4]), (0, 0, 0), 2)

    return img

# Оси координат
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255, 0, 0), 5)  # Ось X (синий)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0, 255, 0), 5)  # Ось Y (зеленый)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0, 0, 255), 5)  # Ось Z (красный)
    return img


circle_coords = np.array([
    [449, 782],
    [519, 775],
    [599, 765],
    [449, 693],
    [514, 678],
    [594, 663],
    [449, 607],
    [514, 589],
    [590, 566]
], dtype=np.float32)

objp = np.zeros((3 * 3, 3), np.float32)
objp[:, :2] = np.mgrid[0:3, 0:3].T.reshape(-1, 2)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
retval, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, circle_coords, camera_matrix, dist_coeffs)


# Список центров пирамид отн-но НК
base_centers = [
    [-0.5, 0.2, 0],
    [0.0, -0.3, -3.0],
    [-0.5, -0.8, 0],
    [0.0, -0.8, -1.7]
]
base_size = 0.4 
height = 1.0 

# Отрисовка пирамид
for base_center in base_centers:
    img = draw_pyramid(img, base_center, base_size, height, rvecs, tvecs, camera_matrix, dist_coeffs)

imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

img = draw(img, circle_coords, imgpts)

cv2.imshow('Pyramids with Axes on Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


